# L=2 BMN SU2 matrix model

In [1]:
import numpy as np
import pylab
import matplotlib.pyplot as plt
import qiskit
from qiskit.quantum_info import Pauli
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit_algorithms import NumPyEigensolver
import time
import sys
sys.path.append('../utility')
from vqe_run import *
from qc_ansatze import *

# Load H

In [2]:
def open_n_process(file_link):
    file = open(file_link, "r")
    content = file.read()

    data = content.split(',\n')
    data[0] = data[0].split('\n')[1]
    data[-1] = data[-1].split('\n')[0]

    data_coeff = []
    data_str = []
    for i in range(len(data)):
        data_coeff.append(float(data[i].split('*')[0]))
        data_str.append(data[i].split('*')[1].split(' ')[1])
    return data_coeff, data_str
    
def check_min_max(data_coeff):
    print(f'Min absolute value is {np.round(np.min(np.abs(data_coeff[1:])),6)}')
    print(f'Max absolute value is {np.round(np.max(np.abs(data_coeff[1:])),5)}')
    print(f'Mean absolute value is {np.round(np.mean(np.abs(data_coeff[1:])),5)}')
    
def get_k_largest_ops(data_coeff, data_str, k):
    ind_list = np.argsort(np.abs(data_coeff[1:]))[-k:]
    ops = [data_str[1:][i] for i in ind_list]
    val= [np.round(data_coeff[1:][i],4) for i in ind_list]
    return ind_list, ops, val

In [3]:
gg = '20'
data_coeff, data_str = open_n_process(f"../utility/L2_BMN_l{gg}_Hpauli.txt")

In [4]:
Hpauli = list(zip(data_str, data_coeff))
H4q = SparsePauliOp.from_list(Hpauli)
# exactly diagonalize the system using numpy routines
solver = NumPyEigensolver(k=4)
exact_solution = solver.compute_eigenvalues(H4q)
#print("Exact Result of qubit hamiltonian:", np.real(exact_solution.eigenvalues))
E_exact = np.round(np.real(exact_solution.eigenvalues)[0],6)
print(f'{check_min_max(data_coeff)}')
print(f'E_exact = {E_exact}')

Min absolute value is 0.5
Max absolute value is 0.75
Mean absolute value is 0.53125
None
E_exact = 0.08385


In [5]:
Hpauli

[('IIIIIIIII', 6.75),
 ('ZIIIIIIII', -0.5),
 ('IZIIIIIII', -0.5),
 ('IIZIIIIII', -0.5),
 ('IIIZIIIII', -0.5),
 ('IIIIZIIII', -0.5),
 ('IIIIIZIII', -0.5),
 ('IIIIIIZII', -0.75),
 ('IIIIIIIZI', -0.75),
 ('IIIIIIIIZ', -0.75),
 ('XXIXXIIII', -0.49999999999999983),
 ('XIXXIXIII', -0.49999999999999983),
 ('IXXIXXIII', -0.49999999999999983),
 ('IIXIIIYXI', 0.4999999999999999),
 ('IIXIIIXYI', 0.4999999999999999),
 ('IIIIIXXXI', 0.4999999999999999),
 ('IIIIIXYYI', -0.4999999999999999),
 ('IXIIIIYZX', -0.4999999999999999),
 ('IXIIIIXZY', -0.4999999999999999),
 ('IIIIXIXZX', -0.4999999999999999),
 ('IIIIXIYZY', 0.4999999999999999),
 ('XIIIIIIYX', 0.4999999999999999),
 ('XIIIIIIXY', 0.4999999999999999),
 ('IIIXIIIXX', 0.4999999999999999),
 ('IIIXIIIYY', -0.4999999999999999)]

# EvolvedOperatorAnsatz

In [5]:
ind15, ops15, vals15 = get_k_largest_ops(data_coeff, data_str, 15)
ind20, ops20, vals20 = get_k_largest_ops(data_coeff, data_str, 20)

ops_Hp15 =[Pauli(f'{ops15[i]}') for i in range(len(ops15))]
ops_Hp20 =[Pauli(f'{ops20[i]}') for i in range(len(ops20))]
ops_H =[Pauli(f'{data_str[1:][i]}') for i in range(len(data_str[1:]))]

In [6]:
ev_op_Hp15 = EvolvedOperatorAnsatz(ops_Hp15, reps=1, insert_barriers=True)
ev_op_Hp20 = EvolvedOperatorAnsatz(ops_Hp20, reps=1, insert_barriers=True)
ev_op_H = EvolvedOperatorAnsatz(ops_H, reps=1, insert_barriers=True)

ev_op_Hp15_2f = EvolvedOperatorAnsatz(ops_Hp15, reps=2, insert_barriers=True)
ev_op_Hp20_2f = EvolvedOperatorAnsatz(ops_Hp20, reps=2, insert_barriers=True)
ev_op_H_2f = EvolvedOperatorAnsatz(ops_H, reps=2, insert_barriers=True)

ev_op_Hp15_3f = EvolvedOperatorAnsatz(ops_Hp15, reps=3, insert_barriers=True)
ev_op_Hp20_3f = EvolvedOperatorAnsatz(ops_Hp20, reps=3, insert_barriers=True)
ev_op_H_3f = EvolvedOperatorAnsatz(ops_H, reps=3, insert_barriers=True)

ev_op_Hp15_4f = EvolvedOperatorAnsatz(ops_Hp15, reps=4, insert_barriers=True)
ev_op_Hp20_4f = EvolvedOperatorAnsatz(ops_Hp20, reps=4, insert_barriers=True)
ev_op_H_4f = EvolvedOperatorAnsatz(ops_H, reps=4, insert_barriers=True)

ansatz_list = [ev_op_Hp15, ev_op_Hp20, ev_op_H, 
               ev_op_Hp15_2f, ev_op_Hp20_2f, ev_op_H_2f,
               ev_op_Hp15_3f, ev_op_Hp20_3f, ev_op_H_3f,
               ev_op_Hp15_4f, ev_op_Hp20_4f, ev_op_H_4f]

ansatz_names = ['ev_op_Hp15', 'ev_op_Hp20', 'ev_op_H', 
               'ev_op_Hp15_2f','ev_op_Hp20_2f','ev_op_H_2f',
               'ev_op_Hp15_3f','ev_op_Hp20_3f','ev_op_H_3f',
               'ev_op_Hp15_4f','ev_op_Hp20_4f','ev_op_H_4f']

print(f'number of params: {[ansatz_list[i].num_parameters for i in range(len(ansatz_list))]}')

number of params: [15, 20, 24, 30, 40, 48, 45, 60, 72, 60, 80, 96]


In [8]:
print(*list(zip(ops20, vals20)), sep = '\n')

('XIIIIIIYX', 0.5)
('IIIIXIYZY', 0.5)
('IIIIXIXZX', -0.5)
('IXIIIIXZY', -0.5)
('IXIIIIYZX', -0.5)
('IIIIIXYYI', -0.5)
('IIIIIXXXI', 0.5)
('IIXIIIXYI', 0.5)
('IIIXIIIYY', -0.5)
('IIIXIIIXX', 0.5)
('IIXIIIYXI', 0.5)
('IIIIIZIII', -0.5)
('IIIIZIIII', -0.5)
('IIIZIIIII', -0.5)
('IIZIIIIII', -0.5)
('IZIIIIIII', -0.5)
('ZIIIIIIII', -0.5)
('IIIIIIIIZ', -0.75)
('IIIIIIIZI', -0.75)
('IIIIIIZII', -0.75)


In [9]:
print(*list(zip(ops15, vals15)), sep = '\n')

('IIIIIXYYI', -0.5)
('IIIIIXXXI', 0.5)
('IIXIIIXYI', 0.5)
('IIIXIIIYY', -0.5)
('IIIXIIIXX', 0.5)
('IIXIIIYXI', 0.5)
('IIIIIZIII', -0.5)
('IIIIZIIII', -0.5)
('IIIZIIIII', -0.5)
('IIZIIIIII', -0.5)
('IZIIIIIII', -0.5)
('ZIIIIIIII', -0.5)
('IIIIIIIIZ', -0.75)
('IIIIIIIZI', -0.75)
('IIIIIIZII', -0.75)


# VQE

In [7]:
seed = 170
iterations = 2000
algorithm_globals.random_seed = seed

#estimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)
#storing values
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
def run_qve_w_specified_optimizer(iterations, optimizer, ansatz):
    opt = optimizer(maxiter = iterations)
    vqe = VQE(noiseless_estimator, ansatz, optimizer=opt, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H4q).eigenvalue.real
    print(f"VQE result: {result:.6f}")
    return result

# COBYLA

At step 7, with ev_op_Hp20_3f
VQE result: 0.625000
Length of this optimization 1077, time taken = 266.295 

In [11]:
r_cobyla=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(iterations, COBYLA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_cobyla.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))
    

At step 0, with ev_op_Hp15
VQE result: 2.877441
Length of this optimization 179, time taken = 20.638 

At step 1, with ev_op_Hp20
VQE result: 0.670410
Length of this optimization 262, time taken = 34.059 

At step 2, with ev_op_H
VQE result: 3.514160
Length of this optimization 247, time taken = 37.565 

At step 3, with ev_op_Hp15_2f
VQE result: 0.923828
Length of this optimization 339, time taken = 40.483 

At step 4, with ev_op_Hp20_2f
VQE result: 3.046387
Length of this optimization 548, time taken = 108.271 

At step 5, with ev_op_H_2f
VQE result: 3.053711
Length of this optimization 581, time taken = 137.534 

At step 6, with ev_op_Hp15_3f
VQE result: 0.883789
Length of this optimization 500, time taken = 79.81 

At step 7, with ev_op_Hp20_3f
VQE result: 0.625000
Length of this optimization 1077, time taken = 288.645 

At step 8, with ev_op_H_3f
VQE result: 3.803223
Length of this optimization 953, time taken = 321.402 

At step 9, with ev_op_Hp15_4f
VQE result: 0.959473
Length of

In [12]:
df1 = pd.concat([r_cobyla[i] for i in range(len(r_cobyla))], axis = 1)
df1.to_csv(f'L2_l{gg}_op_ev_cobyla.csv')

# SPSA

At step 1, with ev_op_Hp20
VQE result: 0.374023
Length of this optimization 3551, time taken = 358.019 

At step 1, with ev_op_Hp20
VQE result: 3.061523
Length of this optimization 4051, time taken = 419.363 

In [13]:
r_spsa=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(iterations, SPSA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_spsa.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15
VQE result: 0.872070
Length of this optimization 4051, time taken = 262.922 

At step 1, with ev_op_Hp20
VQE result: 3.061523
Length of this optimization 4051, time taken = 419.363 

At step 2, with ev_op_H
VQE result: 1.405273
Length of this optimization 4051, time taken = 466.275 

At step 3, with ev_op_Hp15_2f
VQE result: 0.881348
Length of this optimization 4051, time taken = 362.397 

At step 4, with ev_op_Hp20_2f
VQE result: 0.417480
Length of this optimization 4051, time taken = 597.762 

At step 5, with ev_op_H_2f
VQE result: 1.178223
Length of this optimization 4051, time taken = 705.155 

At step 6, with ev_op_Hp15_3f
VQE result: 0.907715
Length of this optimization 4051, time taken = 467.068 

At step 7, with ev_op_Hp20_3f
VQE result: 0.825684
Length of this optimization 4051, time taken = 773.939 

At step 8, with ev_op_H_3f
VQE result: 2.179688
Length of this optimization 4051, time taken = 919.236 

At step 9, with ev_op_Hp15_4f
VQE result: 0.93

In [14]:
df2 = pd.concat([r_spsa[i] for i in range(len(r_spsa))], axis = 1)
df2.to_csv(f'L2_l{gg}_op_ev_spsa.csv')

## Retrain best ansatz

At step 4, with ev_op_Hp20_2f
VQE result: 0.417480
Length of this optimization 4051, time taken = 597.762 

In [9]:
#Increasing the iters makes it better
iterations = 2500
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20_2f)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_a = counts
values_a = values

VQE result: 0.341797
Length of this optimization 5051, time taken = 971.087 



In [10]:
iterations = 3000
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20_2f)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_b = counts
values_b = values

VQE result: 0.383301
Length of this optimization 6051, time taken = 1043.033 



In [11]:
iterations = 1500
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_c = counts
values_c = values

VQE result: 3.046875
Length of this optimization 3051, time taken = 334.733 



In [12]:
iterations = 1750
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_c = counts
values_c = values

VQE result: 0.205078
Length of this optimization 3551, time taken = 352.396 



In [14]:
df = pd.DataFrame({'ev_op_Hp20': values_c})
df.to_csv(f'results/L2_l{gg}_op_ev_spsa_best.csv')
df

,ev_op_Hp20
0,6.109863
1,7.094238
2,6.571777
3,7.173340
4,7.361816
...,...
3546,0.535645
3547,0.597168
3548,1.136230
3549,1.218750


# SLSQP

In [15]:
r_slsqp=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(iterations, SLSQP, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_slsqp.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15
VQE result: 7.366699
Length of this optimization 16, time taken = 2.473 

At step 1, with ev_op_Hp20
VQE result: 6.605469
Length of this optimization 21, time taken = 3.934 

At step 2, with ev_op_H
VQE result: 7.176758
Length of this optimization 25, time taken = 3.816 

At step 3, with ev_op_Hp15_2f
VQE result: 5.892578
Length of this optimization 31, time taken = 3.709 

At step 4, with ev_op_Hp20_2f
VQE result: 6.049805
Length of this optimization 41, time taken = 8.839 

At step 5, with ev_op_H_2f
VQE result: 6.765625
Length of this optimization 49, time taken = 12.467 

At step 6, with ev_op_Hp15_3f
VQE result: 5.543945
Length of this optimization 46, time taken = 7.2 

At step 7, with ev_op_Hp20_3f
VQE result: 7.210449
Length of this optimization 61, time taken = 18.351 

At step 8, with ev_op_H_3f
VQE result: 6.566895
Length of this optimization 73, time taken = 25.169 

At step 9, with ev_op_Hp15_4f
VQE result: 5.532715
Length of this optimization 61

In [16]:
df3 = pd.concat([r_slsqp[i] for i in range(len(r_slsqp))], axis = 1)
df3.to_csv(f'L2_l{gg}_op_ev_slsqp.csv')